# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install --upgrade azureml-sdk[notebooks,automl] -q
%pip install azureml-widgets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires pandas>=1.4.0, but you have pandas 1.3.5 which is incompatible.
dask-expr 1.1.14 requires dask==2024.9.0, but you have dask 2023.2.0 which is incompatible.
dask-expr 1.1.14 requires pandas>=2, but you have pandas 1.3.5 which is incompatible.
azureml-automl-dnn-nlp 1.57.0 requires torch==1.13.1, but you have torch 2.4.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pa

In [2]:


import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
session_id = "268861"
subscription_id= "d7f39349-a66b-446e-aba6-0053c2cf1c11"
resource_group= f"aml-quickstarts-{session_id}"
workspace_name= f"quick-starts-ws-{session_id}"

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'automl-exp'
project_folder = './capstone'
experiment=Experiment(ws, experiment_name)
experiment

quick-starts-ws-268861
aml-quickstarts-268861
westus2
d7f39349-a66b-446e-aba6-0053c2cf1c11


Name,Workspace,Report Page,Docs Page
automl-exp,quick-starts-ws-268861,Link to Azure Machine Learning studio,Link to Documentation


In [5]:


# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/hknguyen20/nd00333-capstone/refs/heads/master/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
df = dataset.to_pandas_dataframe()
df.head()


{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,False,Normal,172,False,0.0,Up,False
1,49,F,NAP,160,180,False,Normal,156,False,1.0,Flat,True
2,37,M,ATA,130,283,False,ST,98,False,0.0,Up,False
3,48,F,ASY,138,214,False,Normal,108,True,1.5,Flat,True
4,54,M,NAP,150,195,False,Normal,122,False,0.0,Up,False


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_v2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [22]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             test_size=0.2,
                             label_column_name="HeartDisease",   
                             path = project_folder,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [23]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_0962bc2d-562e-4cb9-8001-1f1ef1747521,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [24]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_0962bc2d-562e-4cb9-8001-1f1ef1747521,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number o

{'runId': 'AutoML_0962bc2d-562e-4cb9-8001-1f1ef1747521',
 'target': 'compute-cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2024-10-19T10:20:20.877523Z',
 'endTimeUtc': '2024-10-19T10:37:18.9365Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster-1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-exp","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-268861","workspace_name":"quick-starts-ws-268861","region":"westus2","compute_target":"compute-cluster

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [25]:
best_run, fitted_model = automl_run.get_output()
print(best_run)

Run(Experiment: automl-exp,
Id: AutoML_0962bc2d-562e-4cb9-8001-1f1ef1747521_46,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
print(fitted_model)

Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook268861/code/Users/odl_user_268861')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('28', Pipeli...roblem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0, reg_lambda=1.7708333333333335, subsample=0.9, tree_method='auto'))]))], flatten_transform=False, weights=[0.2222222222222222, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]))])
Y_transformer(['LabelEncoder', LabelEncoder()])


In [29]:
best_run.get_metrics()

{'f1_score_micro': 0.8828767123287671,
 'recall_score_weighted': 0.8828767123287671,
 'f1_score_macro': 0.8779237393916282,
 'weighted_accuracy': 0.887113388521524,
 'recall_score_micro': 0.8828767123287671,
 'log_loss': 0.3632039746956787,
 'accuracy': 0.8828767123287671,
 'precision_score_micro': 0.8828767123287671,
 'precision_score_macro': 0.8839000734544171,
 'recall_score_macro': 0.8778328166602754,
 'AUC_macro': 0.9353332031941155,
 'norm_macro_recall': 0.7556656333205508,
 'average_precision_score_micro': 0.9369677544423404,
 'average_precision_score_macro': 0.9354582269746249,
 'f1_score_weighted': 0.8825043542611134,
 'AUC_weighted': 0.9353332031941157,
 'AUC_micro': 0.9367263190878747,
 'matthews_correlation': 0.761576390049642,
 'precision_score_weighted': 0.8877698088880146,
 'balanced_accuracy': 0.8778328166602754,
 'average_precision_score_weighted': 0.9395579598337944,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_0962bc2d-562e-4cb9-8001-1f1ef1747521_

In [28]:
#TODO: Save the best model
best_run.register_model(model_path="./outputs/", model_name='best_model.pkl', properties={'accuracy':best_run.get_metrics()["accuracy"]})

Model(workspace=Workspace.create(name='quick-starts-ws-268861', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-268861'), name=best_model.pkl, id=best_model.pkl:5, version=5, tags={}, properties={'accuracy': '0.8828767123287671'})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_run.download_file('./outputs/scoring_file_v_1_0_0.py', 'scoring.py')
best_run.download_file('./outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
